## **bronze-companies-stock-news**
#### <p>This notebook extracts and ingests raw stock market news data related to publicly listed companies from Yahoo Finance. It captures headlines, timestamps, and brief summaries to build a foundational news dataset for sentiment or trend analysis.</p>

#### **Importing necessary libraries**

In [26]:
import yfinance as yf
import pandas as pd
import json
import datetime

StatementMeta(, d259ae1a-8582-40d4-98ba-dbc281d4fe99, 30, Finished, Available, Finished)

#### **Variables**

In [ ]:
file_name = "comp_news_data"
layer_name = "bronze"

#### **Step 1: Load the company codes (tickers) from the CSV**

In [27]:
codes = spark.read.csv('Files/master_files/company_code.csv', header=True)
short_codes = [
    row['short_code'].strip()
    for row in codes.select('short_code').distinct().collect()
    if row['short_code']
]

StatementMeta(, d259ae1a-8582-40d4-98ba-dbc281d4fe99, 31, Finished, Available, Finished)

#### **Step 2: Fetch stock news for each ticker**

In [28]:
selected_columns = ['content.title', 'content.provider.displayName', 'content.contentType', 
                    'content.displayTime','content.canonicalUrl.url', 'content.summary']
output = []

for i in short_codes:
    try:
        tickers = yf.Ticker(i)
        news = tickers.news
        df = pd.json_normalize(news)
        df = df[selected_columns]

        if not df.empty:
            df['Company'] = i
            output.append(df)
    except Exception as e:
        print(f"An error occurred for {i}: {e}")


StatementMeta(, d259ae1a-8582-40d4-98ba-dbc281d4fe99, 32, Finished, Available, Finished)

#### **Step 3: Combine all DataFrames and save for Bronze layer**

In [29]:
if output:
    final_com_news_df = pd.concat(output)
    final_com_news_df = spark.createDataFrame(final_com_news_df, ['title', 'displayName', 'contentType', 'displayTime','url', 'summary','Company'])

    final_com_news_df.write.mode("overwrite").partitionBy('Company').parquet(f"Files/{layer_name}/{file_name}")

    print(f"[SUCCESS] Stock news data written to Lakehouse")
else:
    print("[WARNING] No data to write.")

StatementMeta(, d259ae1a-8582-40d4-98ba-dbc281d4fe99, 33, Finished, Available, Finished)

[SUCCESS] Stock news data written to Lakehouse path: Files/bronze/comp_news_data
